In [1]:
#SNEHA -- START -- IMPORT CLEAN DATA FROM RONDA'S EXPORT

In [18]:
#Import Libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import linear_model as lm
import seaborn as sns
import numpy as np
import scipy.stats as stats
from pathlib import Path
import requests
import json
import time
from pprint import pprint
# from api_key import api_key

In [19]:
# CLEANED DATA IMPORT FROM RONDA"S EXPOPRTED CSV
csv_path = Path('Coffee_exp.csv')
coffee_df = pd.read_csv(csv_path, encoding="utf8")
coffee_df.columns

Index(['Unnamed: 0', 'Species', 'Owner', 'Country_of_Origin', 'Farm.Name',
       'Altitude', 'Region', 'Number.of.Bags', 'Bag.Weight', 'Harvest.Year',
       'Grading.Date', 'Variety', 'Processing_Method', 'Aroma', 'Flavor',
       'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean.Cup',
       'Sweetness', 'Cupper.Points', 'CoffeeScore', 'Moisture',
       'Category.One.Defects', 'Quakers', 'Color', 'Category.Two.Defects',
       'unit_of_measurement', 'altitude_low_meters', 'altitude_high_meters',
       'altitude_mean_meters'],
      dtype='object')

In [20]:
coffee_df_new = coffee_df.rename(columns={'Total.Cup.Points': 'CoffeeScore',
                                            'Country.of.Origin': 'Country_of_Origin',
                                         'Processing.Method': 'Processing_Method'})

In [ ]:
#SNEHA -- END -- IMPORT CLEAN DATA FROM RONDA"S EXPORT

In [ ]:
#TAYLOR -- START -- QUESTION 1 : The distribution scores across metrics -Compare Arabica and Robusta (Maybe stacked bar?)

In [ ]:
#TAYLOR -- END -- QUESTION 1 : The distribution scores across metrics -Compare Arabica and Robusta (Maybe stacked bar?)

In [ ]:
#SNEHA -- START -- QUESTION 2 : This team likes Columbian coffee: H0: Distribution of "Good" Scores across metrics is not significant in Columbia vs the world

In [21]:
#Creating High/Average/Low values for each coffee metric score
def Score_categories(DataFrame, Score_Columns): 
    for i, m in enumerate(Score_Columns):
        percentiles =DataFrame[m].quantile([(1/3),(2/3)])

        conditions = [(DataFrame[m] <= (1/3)),(DataFrame[m] > (1/3)) & (DataFrame[m] <= (2/3)),(DataFrame[m] > (2/3))]
        values = ['Low', 'Average', 'High']
        DataFrame[f"{m}_Bucket"] = np.select(conditions, values)

metrics = ['Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body',
'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points',
'CoffeeScore', 'Moisture', 'Category.One.Defects', 'Quakers', 'Category.Two.Defects','altitude_mean_meters','Number.of.Bags']
Score_categories(coffee_df_new, metrics)

In [22]:
coffee_df_new.head()

,Unnamed: 0,Species,Owner,Country_of_Origin,Farm.Name,Altitude,Region,Number.of.Bags,Bag.Weight,Harvest.Year,...,Clean.Cup_Bucket,Sweetness_Bucket,Cupper.Points_Bucket,CoffeeScore_Bucket,Moisture_Bucket,Category.One.Defects_Bucket,Quakers_Bucket,Category.Two.Defects_Bucket,altitude_mean_meters_Bucket,Number.of.Bags_Bucket
0,0,Arabica,metad plc,Ethiopia,metad plc,1950-2200,Guji,300,60 kg,2014,...,High,High,High,High,Low,Low,Low,Low,High,High
1,1,Arabica,metad plc,Ethiopia,metad plc,1950-2200,Guji,300,60 kg,2014,...,High,High,High,High,Low,Low,Low,High,High,High
2,2,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",1600 - 1800 m,NaN,5,1,NaN,...,High,High,High,High,Low,Low,Low,Low,High,High
3,3,Arabica,yidnekachew dabessa,Ethiopia,yidnekachew dabessa coffee plantation,1800-2200,oromia,320,60 kg,2014,...,High,High,High,High,Low,Low,Low,High,High,High
4,4,Arabica,metad plc,Ethiopia,metad plc,1950-2200,Guji,300,60 kg,2014,...,High,High,High,High,Low,Low,Low,High,High,High


In [54]:
#High score distribution for Colombia
metrics = ['Aroma_Bucket', 'Flavor_Bucket','Aftertaste_Bucket', 'Acidity_Bucket', 'Body_Bucket', 'Balance_Bucket','Uniformity_Bucket', 'Clean.Cup_Bucket', 'Sweetness_Bucket',\
           'Cupper.Points_Bucket', 'CoffeeScore_Bucket', 'Moisture_Bucket','Category.One.Defects_Bucket', 'Quakers_Bucket','Category.Two.Defects_Bucket', 'altitude_mean_meters_Bucket',\
           'Number.of.Bags_Bucket']
colombia_dist =[]

def score_counts(cols):
    for i,m in enumerate(cols):
        colombia_dist.append(coffee_df_new.loc[(coffee_df_new["Country_of_Origin"]=="Colombia") & (coffee_df_new[m]=="High"),m].count())
                                
score_counts(metrics)
colombia_dist = [i for i in colombia_dist if i!=0]
colombia_dist

[173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 34, 10, 123, 150, 172]

In [55]:
#High score distribution for rest of the world (coffee_df_new["Country_of_Origin"]!="Colombia") & 
metrics = ['Aroma_Bucket', 'Flavor_Bucket','Aftertaste_Bucket', 'Acidity_Bucket', 'Body_Bucket', 'Balance_Bucket','Uniformity_Bucket', 'Clean.Cup_Bucket', 'Sweetness_Bucket',\
           'Cupper.Points_Bucket', 'CoffeeScore_Bucket', 'Moisture_Bucket','Category.One.Defects_Bucket', 'Quakers_Bucket','Category.Two.Defects_Bucket', 'altitude_mean_meters_Bucket',\
           'Number.of.Bags_Bucket']
All_dist =[]

def score_counts(cols):
    for i,m in enumerate(cols):
        All_dist.append(coffee_df_new.loc[(coffee_df_new[m]=="High"),m].count())                           
score_counts(metrics)

All_dist_norm=(All_dist/sum(All_dist))*sum(colombia_dist)
All_dist_round = [int(round(i,0)) for i in All_dist_norm]
All_dist_round = [i for i in All_dist_round if i!=0]
All_dist_round

[173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 26, 11, 127, 152, 173]

In [56]:
# With 17 rows, the degree of freedom is 17-1 = 16
# With a p-value of 0.05, the confidence level is 1.00-0.05 = 0.95.
critical_value = stats.chi2.ppf(q = 0.95, df = 3)
critical_value

7.814727903251179

In [57]:
stats.chisquare(colombia_dist, All_dist_round)

Power_divergenceResult(statistic=2.71052794071055, pvalue=0.9997870353363133)

In [ ]:
#SNEHA -- END -- QUESTION 2 : This team likes Columbian coffee: H0: Distribution of "Good" Scores across metrics is not significant in Columbia vs the world

In [ ]:
#GANESH -- START -- QUESTION 3 : Procesing Method - Visualize with scatter - metrics Scores vs Processing Method

In [ ]:
#GANESH -- END -- QUESTION 3 : Procesing Method - Visualize with scatter - metrics Scores vs Processing Method

In [ ]:
#TAYLOR -- START -- QUESTION 4 : ANOVA - Flavor, Aroma, Body - Distribution impact by Processing methods

In [ ]:
#TAYLOR -- END -- QUESTION 4 : ANOVA - Flavor, Aroma, Body - Distribution impact by Processing methods

In [ ]:
#AARON -- START -- QUESTION 5 : Color coded map for Species? Overall Scores?

In [ ]:
countries = ["Spain"]#, "England", "Ireland"]
url = f"https://api.geoapify.com/v1/geocode/search?"
country_coordinates = []
def color_coffee_map(countries):
    
     #coffee_df_new["Country"]
    for i, country in enumerate(countries):

        
        #target_country = countries #coffee_df_new["Country"]
        params = {'country' : country,
                  'apiKey': api_key,
                  }
        response = requests.get(url, params=params).json()
        print(response.status_code)
        print(json.dumps(response, indent=4, sort_keys=True))
        
        latitude = response["features"][0]["properties"]["lat"]
        longitude = response["features"][0]["properties"]["lon"]

        country_coordinates.append({
            "country" : country,
            "lat" : latitude,
            "lon" : longitude
        })
        #else: pass

        country_coordinates_df = pd.DataFrame(country_coordinates)
        print(country_coordinates_df)

        #add lat and long columns into df

    map = country_coordinates_df.hvplot.points(
            "lon",
            "lat",
            geo = True,
            tiles = "OSM",
            
            #size = "CoffeeScore",
            
            color = "country",
            frame_width = 800,
            frame_height = 600,
            scale = 0.5

           )
    return map


color_coffee_map(countries)

In [ ]:
#AARON -- END -- QUESTION 5 : Color coded map for Species? Overall Scores?